In [1]:
import random

def generate_maze(h, w, start_x, start_y):
    maze = [['.' for _ in range(w)] for _ in range(h)]
    maze[start_x][start_y] = 'S'

    # 保留至少一条路径
    current_x = start_x
    current_y = start_y
    while True:
        direction = random.choice(['up', 'down', 'left', 'right'])
        if direction == 'up' and current_x > 0:
            current_x -= 1
        elif direction == 'down' and current_x < h - 1:
            current_x += 1
        elif direction == 'left' and current_y > 0:
            current_y -= 1
        elif direction == 'right' and current_y < w - 1:
            current_y += 1

        if maze[current_x][current_y] == '.':
            maze[current_x][current_y] = '.'
        else:
            break

    # 随机放置终点
    end_x, end_y = current_x, current_y
    while (end_x, end_y) == (start_x, start_y):
        end_x = random.randint(0, h - 1)
        end_y = random.randint(0, w - 1)
    maze[end_x][end_y] = 'E'

    # 随机放置墙壁
    for i in range(h):
        for j in range(w):
            if maze[i][j] == '.' and random.random() < 0.3:
                maze[i][j] = 'W'

    return maze

h = 30  # 迷宫高度
w = 30  # 迷宫宽度
start_x = random.randint(0, h - 1)
start_y = random.randint(0, w - 1)

maze = generate_maze(h, w, start_x, start_y)
print(start_x, start_y)
for row in maze:
    print(' '.join(row))

18 26
. . . W W . . . . . . . W W . . W . . . . . . . . . . . W .
. W . . . . . . W . . . . . . W W . . W . . . . . W . W . .
. . . . W . . . . . W . . . . W . . . W . . . . W W . W . .
. . . . . . . . . W . . W W . . . . . . . W W W . W . W . .
. W . . . W W . . W W W . . . . . W . . . . W . W . W . . .
. . W . . . . W . . . W . . . . . . W W . . . W . . . . . .
. W . . . W . . . . . W W . W . . . . . . . . . . W . . . W
. . W W . W W . . . . . . . W . . W W . . W . . . W W W . .
. . . . . . . . W . . . . W . . . W . . . . W . . . . W . .
. . . . . . W W . . . . . W . . . . . W W W . . . W . . . W
W . W W W . . . . . . W W . W . . W . W W W W W . . W W . .
W . . . . . . W W . . . . W . . . . . . W W W . . . W . . .
. W . . . . . . . W . W . . . . W . W . . W . W . W . . . .
W . W . . . . W . W W . W . . W W . . . W . . W . . . W . .
. W . . . W . . W . . . . . . . W . . . W . W W . . . W . W
. . . . . . . . . . W . . . . . . W . . . W W W . . . . . .
. . W . . . . . . W . . W . . W . 

In [2]:
import numpy as np
from PIL import Image

def dfs(maze, x, y, frames, visited, path, step_count, max_steps):
    if step_count >= max_steps:
        return False

    if maze[x][y] == 'E':
        return True

    visited.add((x, y))
    path.append((x, y))
    maze[x][y] = 'V'
    frames.append(np.copy(maze))

    dx = [-1, 1, 0, 0]
    dy = [0, 0, -1, 1]

    for i in range(4):
        nx = x + dx[i]
        ny = y + dy[i]

        if 0 <= nx < len(maze) and 0 <= ny < len(maze[0]) and maze[nx][ny] != 'W' and (nx, ny) not in visited:
            if dfs(maze, nx, ny, frames, visited, path, step_count + 1, max_steps):
                return True

    frames.append(np.copy(maze))
    maze[x][y] = '.'
    path.pop()
    return False

def generate_frames(maze, max_steps, block_size):
    frames = []
    visited = set()
    path = []
    start_x, start_y = None, None

    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'S':
                start_x, start_y = i, j

    if start_x is None or start_y is None:
        raise ValueError("No start position found in the maze.")

    dfs(maze, start_x, start_y, frames, visited, path, 0, max_steps)

    return frames

def generate_images(frames, block_size, color_map, frame_skip=2):
    images = []

    for i, frame in enumerate(frames):
        if i % frame_skip == 0:
            img = np.zeros((len(frame) * block_size, len(frame[0]) * block_size, 3), dtype=np.uint8)
            for i in range(len(frame)):
                for j in range(len(frame[0])):
                    img[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size] = color_map[frame[i][j]]

            images.append(Image.fromarray(img))

    return images

def save_maze_solution(maze, start_x, start_y, max_steps, block_size=20, duration=100, loop=0):
    color_map = {'W': (0, 0, 0), '.': (255, 255, 255), 'V': (255, 0, 0), 'E': (0, 255, 0)}
    frames = generate_frames(maze, max_steps, block_size)
    images = generate_images(frames, block_size, color_map)

    images[0].save('maze_solution.gif', save_all=True, append_images=images[1:], optimize=False, duration=duration, loop=loop)

    # Print basic information
    num_steps = len(frames) - 1
    can_reach_goal = num_steps <= max_steps
    print("Number of steps to reach the goal:", num_steps)
    print("Can reach the goal within max_steps?", can_reach_goal)

# Test data
Test_maze = maze
save_maze_solution(Test_maze, start_x, start_y, max_steps=1500)


Number of steps to reach the goal: 125
Can reach the goal within max_steps? True
